In [20]:
from typing import Union

from p1_perceptron import Perceptron

## Test Perceptron

Initialiseer een `Perceptron` voor elk van de INVERT-, AND- en OR-poorten en test of ze op de juiste manier werken.

## AND-poort

Een AND-poort activeert alleen als beide input op `1` staan. Beide inputs tellen even zwaar mee. Daarom maken we alle
`weights` gewoon 1 en de `threshold` de som van de weights, dus `2`.

In [17]:
perceptron: Perceptron = Perceptron([1, 1], 2)

Nu kunnen we een lijst maken aan alle mogelijke combinaties van input en de verwachte output.

In [18]:
truth_table = (
    ((0, 0), 0),
    ((0, 1), 0),
    ((1, 0), 0),
    ((1, 1), 1),
)

Nu kunnen we een functie schrijven om de output van de perceptron tegen de waarheidstabel te vergelijken. 

In [19]:
def compare_to_truth_table(table: tuple[tuple[Union[float, int]], int]) -> None:
    for row in table:
        output = perceptron.activate(row[0])

        print(f"Input {row[0]} geeft als output: {output}")

        assert output == row[1]

TypeError: unsupported operand type(s) for |: 'type' and 'type'

En deze vervolgens te gebruiken.

In [ ]:
compare_to_truth_table(truth_table)

NameError: name 'compare_to_truth_table' is not defined

## OR-poort

Een OR-poort activeert als minstens 1 van de inputs op `1` staan. We kunnen ook nu weer alle `weights` gewoon op `1`
zetten en de `threshold` 2 maken.

In [ ]:
perceptron: Perceptron = Perceptron([1, 1], 2)

En vervolgens de truth table van OR te definiëren.

In [ ]:
truth_table = (
    ((0, 0), 0),
    ((0, 1), 1),
    ((1, 0), 1),
    ((1, 1), 1),
)